<a href="https://colab.research.google.com/github/olegchistoprudov0210-ctrl/learning-quantum/blob/main/16_phase_flip_error_correction_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
# --- 1. INITIALIZATION ---
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
H = (1/np.sqrt(2) * np.array([[1, 1], [1, -1]]))
H3 = np.kron(H, np.kron(H, H))
CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

q_sup = np.array([0.707, 0.707])
q_zero = np.array([1, 0])
psi = np.kron(q_sup, np.kron(q_zero, q_zero))
P0 = np.array([[1, 0], [0, 0]])
P1 = np.array([[0, 0], [0, 1]])

# --- 2. ENCODING (Logical Qubit) ---
# Create entanglement and change basis to Phase-basis
psi_CNOT = np.kron(CNOT, I) @ psi
CNOT_02 = np.kron(P0, np.kron(I, I)) + np.kron(P1, np.kron(I, X))
psi_encoded = CNOT_02 @ psi_CNOT
psi_protected = H3 @ psi_encoded

# --- 3. ERROR INJECTION ---
# Simulate a Phase-Flip (Z) error on the 2nd qubit (index 1)
E_phase = np.kron(I, np.kron(Z, I))
psi_noisy = E_phase @ psi_protected

# --- 4. DETECTION (Syndrome Measurement) ---
# Transform back to Z-basis to "see" the phase error as a bit-flip
psi_detect = H3 @ psi_noisy

def checkerror(n):
  idx_list = np.where(np.abs(n) > 0.1)
  first_idx = idx_list[0][0]
  bits = format(first_idx, '03b')
  if bits[0] != bits[1] and bits[0] != bits[2]:
    return 0
  if bits[1] != bits[0] and bits[1] != bits[2]:
    return 1
  if bits[2] != bits[0] and bits[2] != bits[1]:
    return 2

n = checkerror(psi_detect)

# --- 5. RECOVERY ---
# Fix the detected bit-flip and return to Phase-basis
def recovery(v, error_bit):
  if n == 0:
    return np.kron(X, np.kron(I, I)) @ v
  if n == 1:
    return np.kron(I, np.kron(X, I)) @ v
  if n == 2:
    return np.kron(I, np.kron(I, X)) @ v
  else:
    return v

psi_fixed = recovery(psi_detect, n)
psi_final = H3 @ psi_fixed

# --- 6. RESULTS ---
print(f"Phase-Flip error injected into qubit index: 1")
print(f"Noisy State (Phase check):\n{np.round(psi_noisy, 3)}")

print(f"Decoder identified the 'guilty' qubit: {n}")

# Funal result
print(f"Final logical state:\n{np.round(psi_final, 3)}")

Phase-Flip error injected into qubit index: 1
Noisy State (Phase check):
[ 0.5  0.   0.  -0.5  0.   0.5 -0.5  0. ]
Decoder identified the 'guilty' qubit: 1
Final logical state:
[0.5 0.  0.  0.5 0.  0.5 0.5 0. ]
